<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
!pip install censusgeocode
!pip install datatable
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session
import datetime
from datetime import datetime,timedelta
!pip install pytz
import pytz
import datatable as dt
import sys
from rets import Session

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Address

In [82]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()

#filter = {
       # "MlsStatus": "SLD",
       # "ModificationTimestamp": {"$gte": dt_string}
        #}
#result = rets_client.search(resource='Property', resource_class='Property', limit = 10,search_filter=filter)


tz_IN = pytz.timezone('US/Central')
ini_time_for_now = datetime.now(tz_IN)
today_before_1hours = ini_time_for_now - timedelta(hours = 1)
dt_string = today_before_1hours.strftime("%Y-%m-%dT%H:%M:%S.000")

dt_string


'2022-06-22T18:04:42.000'

In [87]:
#BOM
relist_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(StatusChangeTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
di ={}
df = pd.DataFrame()
# not empty value
for item in relist_result:
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and item['StateOrProvince']== 'Texas' and item['PreviousStatus']!='Coming Soon' and item['PreviousStatus']!='Active' and item['PreviousStatus']!=""):
      print(item['StatusChangeTimestamp'])
      df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) 
      di = item 

for key in di:
  lis.append(key)
df = df.T
df.columns = lis
print(len(df.index))
df['FullAddress'] = df['StreetNumber'] + " " + df['StreetName']  + ", " + df['City'] + ", " +df['StateOrProvince'] + " " + df['PostalCode']
df = df[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','FullAddress']]


2022-06-22T18:06:02.990
2022-06-22T18:32:47.133
2022-06-22T18:31:10.423
2022-06-22T18:42:43.403
2022-06-22T18:22:01.633
2022-06-22T18:28:28.477
2022-06-22T18:38:09.913
7


# Result

In [89]:
dfa = df[['FullAddress','ListingId','ListPrice','LivingArea','BathroomsFull','BedroomsTotal','YearBuilt','GarageSpaces','PoolYN','LotSizeAcres','SubdivisionName']]
dfa

,FullAddress,ListingId,ListPrice,LivingArea,BathroomsFull,BedroomsTotal,YearBuilt,GarageSpaces,PoolYN,LotSizeAcres,SubdivisionName
0,"5212 Silver Lake, Plano, Texas 75093",20013218,1399995.00,5888.00,5,5,1997,3,1,0.3800,Lakeside On Preston Ph 4c
1,"415 Mineral Wells, Terrell, Texas 75160",20048115,324900.00,2378.00,2,3,2022,2,0,0.1346,DEWBERRY 2ND Addition
2,"417 Mineral Wells, Terrell, Texas 75160",20048137,324900.00,2378.00,2,3,2022,2,0,0.1346,DEWBERRY 2ND Addition
3,"10202 Deermont, Dallas, Texas 75243",20069122,400000.00,2400.00,2,4,1976,2,1,0.2250,Cimmaron Ridge 01
4,"3212 Spruce, Royse City, Texas 75189",20079969,325000.00,2020.00,2,3,2007,2,0,0.1650,Woodland Creek Ph II
5,"10548 Winding Passage, Fort Worth, Texas 76131",20083707,375000.00,2726.00,3,4,2010,2,0,0.1430,Fossil Hill Estates
6,"4317 Oak Chase, Fort Worth, Texas 76244",20090924,425000.00,1984.00,2,3,2015,2,0,0.1380,Steadman Farms


# Lease


In [65]:
def add(lo_left,lo_right,la_left,la_right):
    a = 0.008 #调整经纬度改这里
    lo_left -= a
    lo_right += a
    la_left -= a
    la_right += a
    return lo_left, lo_right, la_left, la_right

In [66]:
def judgeType(item):
  if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):#改type在这里
    return True

In [67]:
def lease_findifNull(research, lo_left,lo_right,la_left,la_right, bedroom):
  #经纬度查找
  bedroom = int(bedroom)
  filter = {
        "BathroomsTotalInteger": "{}-{}".format(bedroom - 2, bedroom + 1),
        "BedroomsTotal": bedroom,
        "Longitude": "{}-{}".format(lo_left, lo_right),
        "Latitude": "{}-{}".format(la_left, la_right),
        "MlsStatus" : "SLD",
        "CloseDate" : "2022-01-01-2022-05-31"
        }
  result = research.search(resource='Property', resource_class='Property', search_filter=filter)
  lis = []
  count = 0
  di ={}
  df = pd.DataFrame()
  for item in result:
    if(judgeType(item)):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1
            di = item # 存表头
  for key in di:
    lis.append(key)
  if(count >= 5): 
    df = df.T
    df.columns = lis
    return df
  else:
    lo_left, lo_right, la_left, la_right = add(lo_left,lo_right,la_left,la_right)
    return lease_findifNull(research, lo_left,lo_right,la_left,la_right,bedroom)

In [68]:
def lease_findHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,zip,column,bedroom, sub):
  #sub，zip查找，未找到足够的自动转经纬度
  filter = {
        "BathroomsTotalInteger": "{}-{}".format(bedroom - 2, bedroom + 1),
        "BedroomsTotal": bedroom,
        "PostalCode" : zip,
        "MlsStatus" : "SLD",
        "CloseDate" : "2022-01-01-2022-05-31"
        }
  result = research.search(resource='Property', resource_class='Property', search_filter=filter)
  count = 0
  df = pd.DataFrame()
  for item in result: 
    if(judgeType(item)):
      if(sub in item['SubdivisionName']):
         df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
         count += 1       
  if(count >= 5):  
    df = df.T
    df.columns = column # 
    return df
  else:
    return lease_findifNull(research, house_leftLo,house_rightLo, house_leftLa, house_rightLa, bedroom)

In [69]:
def findfirst(result):
  #判断是否有，不用管
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True    

In [133]:
from pandas.core.frame import DataFrame
def insertId(pulled_value, mls,rets_client):
  #mls查找，按照先sub + zip，后geo
  column = []
  for key in pulled_value:
    column.append(key)
  print("\nThis value exists in Website")
  zip = pulled_value.get('PostalCode')
  latitude = pulled_value.get('Latitude')
  longitude = pulled_value.get('Longitude')
  bedroom = pulled_value.get('BedroomsTotal')
  if(zip != "" and latitude != "" and longitude != "" and bedroom != ""):
    if(int(bedroom) >= 3 and int(bedroom) <= 5):
      house_leftLa = float(pulled_value.get('Latitude'))
      house_rightLa = float(pulled_value.get('Latitude')) 
      house_leftLo = float(pulled_value.get('Longitude'))
      house_rightLo = float(pulled_value.get('Longitude'))
      sub = str(pulled_value.get('SubdivisionName').split()[0])
      result1 = lease_findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, rets_client,int(zip) ,column,int(bedroom), sub)        
      result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
      price = []
      cdom_signValue = []
      df_id = pd.DataFrame()
      for item in result1['ClosePrice']:
        item = float(item)
        price.append(item)
      result1['ClosePrice'] = price
      mean1 = result1['ClosePrice'].mean()
      top1 = list(result1['ClosePrice'])
      top1.sort(reverse = True)
      top1_mean = sum(top1[0:3])/3
      ###########################################################
      min1 = list(result1['ClosePrice'])
      min1.sort(reverse = False)
      median = result1['ClosePrice'].median()
      for item in result1['CumulativeDaysOnMarket']:
        item = int(item)
        cdom_signValue.append(item)
      result1['CumulativeDaysOnMarket'] = cdom_signValue
      cdom = result1['CumulativeDaysOnMarket'].mean() 
      df_id['Min','Median','Mean','Top 3 Mean','Max	Avg.','CDOM'] = min1,median,mean1,top1_mean,top1,cdom
      print(df_id)
     #['Min',	'Median',	'Mean',	'Top 3 Mean','Max	Avg.', 'CDOM',	'Section 8']

     # print('The mean for {} is {}'.format(mls, mean1))
     # print('The top 3 for {} are {}'.format(mls, top1[0:3]))
     # print('The average for top 3 {} is {}'.format(mls, top1_mean))
     # result1.to_csv('{}.csv'.format(mls))
      return
    else:
      print("Bedroom number is not qualified")
      return

In [71]:
def insertAdd(number, name, city, state, zip, address, rets_client):
  #add查找，按照先sub + zip，后geo
  pulled_value = {}
  result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number,name,zip))
  for item in result:
    pulled_value = item
  if(len(pulled_value) != 0):
    column = []
    for key in pulled_value:
      column.append(key)
    print("\nThis value exists in Website")
    zip = pulled_value.get('PostalCode')
    latitude = pulled_value.get('Latitude')
    longitude = pulled_value.get('Longitude')
    bedroom = pulled_value.get('BedroomsTotal')
    if(zip != "" and latitude != "" and longitude != "" and bedroom != ""):
      if(int(bedroom) >= 3 and int(bedroom) <= 5):
        house_leftLa = float(pulled_value.get('Latitude'))
        house_rightLa = float(pulled_value.get('Latitude')) 
        house_leftLo = float(pulled_value.get('Longitude'))
        house_rightLo = float(pulled_value.get('Longitude'))
        zip = int(pulled_value.get('PostalCode'))
        sub = str(pulled_value.get('SubdivisionName').split()[0])
        result1 = lease_findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, rets_client,zip,column,int(bedroom), sub)        
        result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result1['ClosePrice']:
          item = float(item)
          price.append(item)
        result1['ClosePrice'] = price
        mean1 = result1['ClosePrice'].mean()
        top1 = list(result1['ClosePrice'])
        top1.sort(reverse = True)
        top1_mean = sum(top1[0:3])/3
        print('The mean for {}, {}, {} is {}'.format(number,name,zip, mean1))
        print('The top 3 for {}, {}, {} are {}'.format(number,name,zip, top1[0:3]))
        print('The average for top 3 {}, {}, {} is {}'.format(number,name,zip, top1_mean))
        result1.to_csv('{} {} {}.csv'.format(number,name,zip))
        return
      else:
        print("Bedroom number is not qualified")
        return
    else:
      print('check coordinates')
      bedroom = 3
      findAdd(rets_client, address, city, state, zip, bedroom)
      return 
  else:
    print('check coordinates')
    bedroom = 3
    findAdd(rets_client, address, city, state, zip, bedroom)
    return 

In [72]:
def findAdd(rets_client, address, city, state, zipcode, bedroom):  
  #geo代码，在mls和address中引用
  result = cg.address(address, city = city, state= state,zipcode = zipcode)
  lo_left = result[0]['coordinates']['x']
  lo_right = result[0]['coordinates']['x']
  la_left = result[0]['coordinates']['y']
  la_right = result[0]['coordinates']['y']
  result = lease_findifNull(rets_client, lo_left, lo_right, la_left, la_right, bedroom)
  
  result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
  price = []
  for item in result['ClosePrice']:
    item = float(item)
    price.append(item)
  result['ClosePrice'] = price
  mean = result['ClosePrice'].mean()
  top = list(result['ClosePrice'])
  top.sort(reverse = True)
  print('The mean for {}, {}, {} is {}'.format(address, state, zip, mean))
  print('The top 3 for {}, {}, {} are {}'.format(address, state, zip, top[0:3]))
  return result.to_csv('{},{},{}.csv'.format(address, state, zip))

In [73]:
def judge(i):
  #不用管，为了美观
    if i % 10 == 1:
        return 'st'
    elif i % 10 == 2:
        return 'nd'
    elif i % 10 == 3:
        return 'rd'
    else:
        return 'th'

In [117]:
def main():
  login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
  username = '0671181_NID'
  password = 'Rt$tg6jx'
  rets_client = Session(login_url, username, password)
  rets_client.login()  

  lis_mls = []
  data = dfa
  for item in data['ListingId']:
    lis_mls.append(int(item))
  for i in range(len(lis_mls)):
    pulled_value = {}
    result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(lis_mls[i]))
    for items in result:
      pulled_value = items
    if(len(pulled_value) != 0):
      insertId(pulled_value, lis_mls[i], rets_client) 
      if(i == len(lis_mls) - 1):
        return


# result

In [ ]:
if __name__ == "__main__":
    print(main()) 
    #5005 Bama Drive, Arlington, Texas 76017

In [74]:
def main():
  login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
  username = '0671181_NID'
  password = 'Rt$tg6jx'
  rets_client = Session(login_url, username, password)
  rets_client.login()

  data = df
  code = []
  name = []
  number = []
  street = []
  city = []
  state = []
  for item in data['FullAdress']:
    a = item.split(',')[0]
    b = item.split(',')[2]
    c = item.split(',')[1]
    street.append(a)
    city.append(c)
    numbers = a.split()[0]
    names = a.split()[1:(len(a.split()) - 1)]
    codes = b.split()[1][0:5]
    states = b.split()[0]
    state.append(states)
    code.append(codes)
    name.append(names)
    number.append(numbers)
  finalN = []
  for item in name:
    string = ""
    for items in item:
      string += items
      string += ' '
    finalN.append(string[0:len(string) - 1])            
  for i in range(len(code)):
    insertAdd(number[i], finalN[i], city[i], state[i], code[i], street[i], rets_client)